In [1]:
import json
import subprocess


In [9]:
def get_video_duration(video_file):
    cmd = ['ffprobe', '-v', 'error', '-show_entries', 'format=duration', '-of', 'default=noprint_wrappers=1:nokey=1', video_file]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)
    duration = float(result.stdout.strip())
    return duration

In [10]:
def extract_clip(video_file, start_time, duration, output_filename):
    command = ['ffmpeg', '-ss', str(start_time), '-i', video_file, '-t', str(duration), '-c', 'copy', output_filename]
    subprocess.run(command)
    print(f"Extracted clip: {output_filename}")

In [8]:
def extract_clips_and_labels(json_file, video_file, output_folder, clip_length=5.0):
    with open(json_file, 'r') as file:
        data = json.load(file)
    for metadata_id, metadata_info in data["metadata"].items():
        start_time, end_time = metadata_info["z"]
        action_labels = '_'.join([label for label in metadata_info["av"].values()])
        duration = end_time - start_time

        # 确定是否需要向前或向后扩展
        if duration < clip_length:
            # 计算额外需要的时间
            extra_time_needed = clip_length - duration
            half_extra_time = extra_time_needed / 2

            # 调整开始和结束时间，以确保视频片段总长度为5秒
            adjusted_start_time = max(0, start_time - half_extra_time)
            adjusted_end_time = adjusted_start_time + clip_length

            # 如果调整后的结束时间超过视频本身的长度，需要向前移动开始和结束时间
            # 假设 total_video_length 是视频的总长度
            if adjusted_end_time > total_video_length:
                adjusted_end_time = min(total_video_length, end_time + half_extra_time)
                adjusted_start_time = adjusted_end_time - clip_length

            output_filename = f"{output_folder}/{metadata_id}_{action_labels}.mp4"
            extract_clip(video_file, adjusted_start_time, clip_length, output_filename)

        else:
            #对于超过5秒的动作，这里可以插入之前讨论的逻辑进行处理